# double alpha decay functions

### specialized functions for double alpha with decay model

In [1]:
def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    tletters = ["A", "B", "C"]
    
    totalLLH = 0 
    for trial in range(0, len(data)):

        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
           
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
            

            # update prob and value for trustee 
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)         
            ev[trustee] = update_value(prob[trustee])
            
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(trustee)
        for x in options:
            prob[x] = prob[x] + decay * (0.5 - prob[x])
            ev[x] =  update_value(prob[x])
    return totalLLH

def model_fit(data):
    
    tries = 100 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [a_gain, a_loss, beta, decay]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 20), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
        if bestFit == 'NA':
            line = ['NA', 'NA', 'NA', 'NA', 'NA']
        else:
            line = np.append(bestFit['x'], bestFit['fun'])
      
    return line